In [3]:
# !pip install python-airtable

  Using cached python_airtable-0.5-py2.py3-none-any.whl (13 kB)


In [4]:
from airtable import Airtable
import pandas as pd

In [14]:
from llama_index.query_engine import PandasQueryEngine
from llama_index.query_engine.pandas_query_engine import PandasQueryEngine

In [6]:
api_key = 'patngX23FWWWnl2iH.e5c8bc51522c7d6db26f4668e80775aa7f344107ec6a1204a5b388427a6e1da7'
base_id = 'appfjmrH6l5m4Z82K'
table_name_mapping = {
    'tblM34kdan10sEZrK': 'items',
    'tblSYZrSYVQJcvNkR': 'customers',
    'tblkGA9TFjiGyiqTN': 'payments',
    'tblm5kBGMQg08mIej': 'suppliers',
    'tblFuLuXFBtrqHbe3': 'catalog',
    'tbly4c5vUFLAP91ga': 'orders'
}


In [7]:
def fetch_all_airtable_data(api_key, base_id, table_name_mapping):
    all_dataframes = {}
    for table_id, table_name in table_name_mapping.items():
        airtable = Airtable(base_id, table_id, api_key)
        records = []
        for record in airtable.get_all():
            records.append(record['fields'])
        df = pd.DataFrame.from_records(records)
        all_dataframes[table_name] = df
        df.to_csv(f'{table_name}.csv', index=False)
    return all_dataframes

In [8]:
def clean_fetched():
    fetch_all_airtable_data(api_key, base_id, table_name_mapping)
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    orders = pd.read_csv('orders.csv')
    pay = pd.read_csv('payments.csv')
    # Use a regular expression to extract digits or NaN
    orders['Num (from Customer)'] = orders['Num (from Customer)'].str.extract(r"(\d+|\d+\.\d+|\d+\.\d+e\+\d+|\d+\.\d+e-\d+|\d+e\+\d+|\d+e-\d+|\d+\.\d+e-\d+|\d+\.\d+e\+\d+|NaN)", expand=False)
    # Convert the 'Extracted_Num' column to numeric (handles NaN)
    orders['Num (from Customer)'] = pd.to_numeric(orders['Num (from Customer)'], errors='coerce')
    orders['Num (from Customer)'] = orders['Num (from Customer)'].astype(pd.Int64Dtype(), errors='ignore')
    orders['Order num'] =pd.to_numeric(orders['Order num'], errors='coerce').astype('float64')
    orders.dropna(subset=['Num (from Customer)'], inplace=True)
    # Use a regular expression to extract digits or NaN
    pay['Num (from Customer)'] = pay['Num (from Customer)'].str.extract(r"(\d+|\d+\.\d+|\d+\.\d+e\+\d+|\d+\.\d+e-\d+|\d+e\+\d+|\d+e-\d+|\d+\.\d+e-\d+|\d+\.\d+e\+\d+|NaN)", expand=False)
    # Convert the 'Extracted_Num' column to numeric (handles NaN)
    pay['Num (from Customer)'] = pd.to_numeric(pay['Num (from Customer)'], errors='coerce')
    pay['Num (from Customer)'] = pay['Num (from Customer)'].astype(pd.Int64Dtype(), errors='ignore')
    pay.sort_values(by=['Num (from Customer)', 'Order # (from Orders)', 'Requested on'], ascending=[True, True, True], inplace=True)
    pay.drop_duplicates(subset=['Num (from Customer)', 'Order # (from Orders)'], keep='last', inplace=True)
    pay['Order # (from Orders)'] =pd.to_numeric(pay['Order # (from Orders)'], errors='coerce').astype('float64')
    
    orders.sort_values(by=['Num (from Customer)', '#', 'Order num','Invoice #'], ascending=[True, True, True, True], inplace=True)
    
    dfa = pd.merge(orders, pay, how='left', left_on=['Num (from Customer)', 'Order num'], right_on=['Num (from Customer)', 'Order # (from Orders)'])
    
    cust = pd.read_csv('customers.csv')
    cust['Num'] = pd.to_numeric(cust['Num'], errors='coerce').astype('Int64')
    cust.dropna(subset=['Name'], inplace=True)
    cust.dropna(subset=['Num'], inplace=True)
    dfa2 = pd.merge(dfa, cust, how='outer', left_on='Num (from Customer)', right_on='Num')
    dfa2['Num (from Customer)'].fillna(dfa2['Num'], inplace=True)
    dfa2['Name (from Customer)'].fillna(dfa2['Name'], inplace=True)
    dfa2['Project name'].fillna(dfa2['Project name (from Orders)'], inplace=True)
    dfa2['Project name (from Orders)'].fillna(dfa2['Project name'], inplace=True)
    dfa2['Invoicing address (from Customer)_x'].fillna(dfa2['Invoicing address'], inplace=True)
    dfa2.drop_duplicates(subset=['Num (from Customer)', 'Order num', 'Project name'], keep='last', inplace=True)
    dfa2.drop_duplicates(subset=['Num (from Customer)','Email (from Manager)','Order total','Created: project name','Date quote sent'], keep='last', inplace=True)
    
    to_drop = ['Manager_x','Today_x','Teams (from Manager)',
           'Team Emails (from Managers) (from Teams) (from Manager)_x',
          'Customer_x','DELETE invoice total','Year','Invoicing contact (from Customer)_x',
           'Invoice button','Items Table_Id','Customer Table_Id',
           'Open URL','Order items Table_Id','Payments Table_Id',
          'Order to supplier Table_Id','Create quote','Payments_button','Paper track',
           'Button quote','Empty order items alert','Check order number',
          'Prod and ship','Ship addresses missing (from Customer)','Create BAT',
          'Check signed BAT','Create mold invoice','Create cost PDF','Alert mold invoice sent',
          'Alert MP invoice payment','Create supplier contract', 'Alert sample approval',
          'Manage invoices', 'Update Customer info','Sample order','MP order',
          '#','Invoice #','Customer name in text', 
            'Quantity (from Shipments) (from Order items)','Remains to be shipped (from Order items)','Invoicing address','Invoice Ref',
            'Customer name in text','Manage BAT','Total on invoice', 'Balance due on invoice',
            'Email (from Invoicing contact) (from Customer)_x','Addresses missing (from Customer)',
            'Today plus 7','Days RS', 'Days since R','Paid for MP downpayment',
            'Paid for MP','Percentage paid','Order items','Total (from Order items)',
            'On invoice (from Order items)','Order unit cost ¥ (from Order items)',
           'Cost check (from Order items)','Price check (from Order items)','Total price on invoice (from Order items)',
            'Total price mold invoice (from Order items)','Picture (from Order items)',
            'Customer contact','Quote PDF','Project type','Payments_x',
            'Amount paid (from Invoices)','Status of payment (from Payments)',
            'Picture for this order (from Order items)','Picture for this order (from Order items) 2',
                        'Status mold payment (from Payments)','Status MP downpayment (from Payments)',
                        'Order photo (from Order items)','Order type',
           'Payments to suppliers','Payment date (from Payments to suppliers)','Supplier (from Payments to suppliers)',
                        'Percentage of downpayment','Balance due time','BAT',
                        'Order to supplier','Inner size','Mold invoice issued on','Mold invoice','Stage (from Payments)',
                        'Shipments (from Order items)','Customer Payment #','Customer_y','Stage','Payments_y','Days to payment due',
                        'Status of payment','Status MP downpayment','Num of days','Warning','Invoicing contact (from Customer)_y',
                        'Invoicing address (from Customer)_y','Email (from Invoicing contact) (from Customer)_y',
                        'Today_y','Invoiced vs. paid','Invoice total (from Orders)','% asked (auto)',
                        'Total cost (from Order items) (from Orders)','¥ due to supplier','Supplier payment status',
                        'Paid to supplier','Wire transfer date (July)','Wire transfer amount','Orders_x',
                        'Order # (from Orders)','Name (from Customer) (from Orders)','Manager (from Orders)',
                        'Order items (from Orders)','View invoice (from Orders)','Supplier payment date (July)','Special discount (from Orders)',
           'Status mold payment','Sales Rep','Contacts','Num','Name','Client','Status_y',
            'Lifetime value','Email (from Invoicing contact)','Invoicing address','Email (from Sales Rep)',
            'Count (Addresses) invoice','Inv addresses missing','Count (Addresses) ship','Ship addresses missing',
            'Count contacts','Contact missing','Email (from Contacts)','Update customer info','Orders_y',
            'Total paid (from Orders)','Manager_y','Team Emails (from Managers) (from Teams) (from Manager)_y',
            'Catalogue','Order status (from Orders)','Shipping addresses', 'Shipping address_y','Other names',
           'Payment requests INV','Website','Addresses','Priority','Type (from Addresses)','Products','Invoice Street name and num (from Addresses)',
            'Invoice City and Zipcode (from Addresses)','Invoice Country (from Addresses)',
          'Container Shipping address #2','Accounting contact Name (from Contacts)','Tax ID / DVT / RFC',
           'Artwork contact Name (from Contacts)','Phone','Quotes','Logistics contact Name (from Contacts)',
           'First contact','Last modified time of first contact']
    dfa2.drop(columns=to_drop, inplace=True)
    dfa2.to_csv('orderspaymentscust.csv', index=False)
    return dfa2

In [13]:
clean_fetched().sample()

,Order num,Quote status,Name (from Customer),Order total,Total paid,Balance due,Email (from Manager),Sales Rep (from Customer),Num (from Customer),Invoicing address (from Customer)_x,Quantity (from Order items),Status (from Customer),Created: project name,Total cost (from Order items),Total paid to suppliers,Total due to suppliers,Balance due to suppliers,Unit price $ (from Order items),Item description (from Order items),Total cost (from Order items) 2,Overview,Email (from Contacts) (from Customer),Total mold invoice,Balance due on mold,Alert quote time,Contact missing (from Customer),Paid for mold,View quote,Update customer info (from Customer),Supplier Overview,Supplier payments,Shipments,Alert do not pay supplier,Alert payment to supplier 2,Check ready date,ETD (from Shipments (containers)) (from Order items) 2,Diff today ready,Alert ETD order,All orders,Balance & shipment,Team lead (from Teams) (from Manager),Email (from Team lead) (from Teams) (from Manager),Order date,Type (from Order items),Types,Order status,Email (from Sales Rep) (from Customer),Project name,Picture (from Catalogue item) (from Order items),Supplier (from Order items),Status (from Sales Rep) (from Customer),Sales Reps,Invoice date,Customer PO number,Invoice PDF,Amount paid (from Payments to suppliers),Timestamp,Ship by,Email (from Sales Reps),Upload supplier invoice,Estimated ready date,Manager (from Order) (from Payments to suppliers),Company name (from Supplier) (from Payments to suppliers),Date quote accepted,Date quote requested,Date quote created,Supplier cost,Sample approval,Date mold invoice sent,Signed BAT,Shipments (containers) (from Order items),ETD (from Shipments (containers)) (from Order items),Internal Rockwood comments,Date quote sent,Alert payment to supplier,Project documents,Special discount,Order special requirements,Commission,Commission paid on,Balance payment paid,Supplier quote requested,Invoices,Please pay,New or repeat,Final balance due,Product details,Shipment details,Shipping address on invoice,Shipping address_x,Country (from Shipping address),City and Zipcode (from Shipping address),Street name and num (from Shipping address),Type (from Shipping address),other date,Amount due,Requested on,Payment due date,Order total (from Orders),Quantity (from Shipments) (from Order items) (from Orders),Balance due (from Orders),Email (from Manager) (from Orders),Num of days until due date,Project name (from Orders),Status_x,View invoice,Payment due in a week,Status (from Sales Rep),Shipping Country (from Addresses),Shipping City and Zipcode (from Addresses),Shipping Street name and num (from Addresses),Purchasing contact Name (from Contacts),First order
867,NaN,NaN,ROMERO AGAVE LLC - AKOYA SPIRITS LLC,NaN,NaN,NaN,NaN,NaN,662,", ,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Active'],NaN,NaN,NaN,NaN,NaN


In [32]:
df = pd.read_csv('orders.csv')

In [33]:
from llama_index.llms import OpenAI
from llama_index import LLMPredictor, PromptHelper, ServiceContext

# Set up the LLM predictor
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# Set up the prompt helper
prompt_helper = PromptHelper(context_window=2048, num_output=256, chunk_overlap_ratio=0.2)

# Create the LLMPredictor object
llm_predictor = LLMPredictor(llm=llm)

# Create the ServiceContext object
service_context = ServiceContext(embed_model=None, node_parser=None, llama_logger=None, callback_manager=None, llm_predictor=llm_predictor, prompt_helper=prompt_helper)

In [34]:
# Create an instance of the PandasQueryEngine class with the service context
pqe = PandasQueryEngine(df=df, service_context=service_context)

In [35]:
# Use the PandasQueryEngine instance to query the Pandas dataframe
results = pqe.query("List the columns with missing values and the number of missing values.")

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 44902 tokens. Please reduce the length of the messages.

In [ ]:
print(results)

In [18]:
response = query_engine.query(
    "Find the customer name who has the biggest balance due",
)
print(response)

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 44900 tokens. Please reduce the length of the messages.

In [ ]:
# get pandas python instructions
print(response.metadata["pandas_instruction_str"])

In [ ]:
eval("df.loc[df['population'].idxmin()]['city']")